## Hybrid Neural Net to solve Regression Problem

We use a neural net with a quantum layer to predict the second half betting lines given the result of the first half and the opening line. The quantum layer is an 8 qubit layer and the model is from Keras. 

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pennylane as qml
import warnings
warnings.filterwarnings('ignore')
tf.keras.backend.set_floatx('float64')

import warnings
warnings.filterwarnings('ignore')

In [16]:
###predict 2nd half line using 1st half total and open ##

df1 = pd.read_csv("nfl_odds.csv")
df1['1H'] = df1['1st'] + df1['2nd']

df2 = pd.read_csv('bet.csv')
df = df1.merge(df2, left_on = 'Team', right_on = 'Tm')

df = df[['1H','Open', 'TO%','PF','Yds','ML', '2H']]
df.head()

,1H,Open,TO%,PF,Yds,ML,2H
0,7,54,9.8,384,6004,375,27
1,10,53.5,9.8,384,6004,290,24
2,21,45,9.8,384,6004,170,24
3,6,3,9.8,384,6004,-170,3
4,10,6.5,9.8,384,6004,-260,2.5


In [4]:
n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

n_layers = 4
weight_shapes = {"weights": (n_layers, n_qubits)}
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [7]:
clayer_1 = tf.keras.layers.Dense(8,  activation="relu")
clayer_2 = tf.keras.layers.Dense(2, activation="relu")
model = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])

In [8]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mae", metrics=["mean_absolute_error"])

In [9]:
df = df[df.Open != 'pk']
df = df[df['2H'] != 'pk']
df['Open'] = df['Open'].astype(float)
df['2H'] = df['2H'].astype(float)
X = df[['1H','Open','TO%','PF','Yds','ML']]
y = df['2H']

X = np.asarray(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [10]:
fitting = model.fit(X_train, y_train, epochs=10, batch_size=5, validation_split=0.15, verbose=2)

2021-07-27 19:36:13.392982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/10
73/73 - 164s - loss: 12.6663 - mean_absolute_error: 12.6663 - val_loss: 12.0027 - val_mean_absolute_error: 12.0027
Epoch 2/10
73/73 - 183s - loss: 10.7188 - mean_absolute_error: 10.7188 - val_loss: 9.4814 - val_mean_absolute_error: 9.4814
Epoch 3/10
73/73 - 187s - loss: 8.1917 - mean_absolute_error: 8.1917 - val_loss: 6.7941 - val_mean_absolute_error: 6.7941
Epoch 4/10
73/73 - 183s - loss: 6.7808 - mean_absolute_error: 6.7808 - val_loss: 5.8536 - val_mean_absolute_error: 5.8536
Epoch 5/10
73/73 - 195s - loss: 6.5286 - mean_absolute_error: 6.5286 - val_loss: 5.8190 - val_mean_absolute_error: 5.8190
Epoch 6/10
73/73 - 186s - loss: 6.4893 - mean_absolute_error: 6.4893 - val_loss: 5.7794 - val_mean_absolute_error: 5.7794
Epoch 7/10
73/73 - 176s - loss: 6.5034 - mean_absolute_error: 6.5034 - val_loss: 5.8553 - val_mean_absolute_error: 5.8553
Epoch 8/10
73/73 - 182s - loss: 6.3533 - mean_absolute_error: 6.3533 - val_loss: 5.7911 - val_mean_absolute_error: 5.7911
Epoch 9/10
73/73 -

In [11]:
X_test = scaler.transform(X_test)
preds = model.predict(X_test)

In [12]:
pred = pd.DataFrame(preds, columns =[ 'prediction1', 'prediction2'])
pred = pred[(pred.prediction1 > 0) & (pred.prediction1 < 30)]
y_test = y_test.reset_index()

In [13]:
y_test = y_test[y_test['2H'] > 6]

compare = pd.concat([pred, y_test], axis=1)
compare = compare.drop('index', axis=1)

In [14]:
compare.dropna()

,prediction1,prediction2,2H
5,22.354296,22.351161,26.5
7,22.354296,22.351161,21.0
8,4.868208,4.572352,7.5
9,22.354296,22.351161,21.0
10,22.354296,22.351161,20.5
20,22.354296,22.351161,24.5
22,22.344846,22.343618,24.0
23,22.354296,22.351161,24.0
27,22.354296,22.351161,27.5
29,22.354296,22.351161,22.5
